# Polygon

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 6.5 MB 30.0 MB/s 
     |████████████████████████████████| 596 kB 20.1 MB/s 
     |████████████████████████████████| 895 kB 37.8 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch

In [ ]:
from transformers import BertForQuestionAnswering

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/781 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

In [ ]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('xlm-roberta-large-qa-multilingual-finedtuned-ru')

In [ ]:
question = 'Что думаешь про фильм Матрица?'
answer_text = 'Матрица отличный фильм, обязательный к просмотру всей семьей!'

In [ ]:
inputs = tokenizer(question, answer_text, return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

In [ ]:
tokenizer.decode(predict_answer_tokens)

'отличный фильм'

In [ ]:
model.config

XLMRobertaConfig {
  "_name_or_path": "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [ ]:
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 25 tokens.


In [ ]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [ ]:
# Run our example through the model.
outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                             token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


# ruGPT3 small

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgraded, 21 newly installed, 0 to remove and 39 not upgraded.
Need to get 85.9 MB of archives.
After this operation, 511 MB of additional d

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 15 kB/s 
     |████████████████████████████████| 12.8 MB 67.1 MB/s 
     |████████████████████████████████| 7.6 MB 28.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torc

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Overwriting setup.sh


In [ ]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for sc

In [ ]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3 MB 11.2 MB/s 
     |████████████████████████████████| 2.9 MB 63.3 MB/s 
     |████████████████████████████████| 1.1 MB 65.9 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96
  Attempting uninstall: transformers
    Found existing installation: transformers 2.8.0
    Uninstalling transformers-2.8.0:
      Successfully uninstalled transformers-2.8.0


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 6.37 MiB/s, done.
Resolving deltas: 100% (410/410), done.


# Generate text (УСПЕШНО)

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated = generate(model, tok, "Фильм Матрица?", num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated[0]

'Фильм Матрица?\nАнатолий Вассерман\n\n\nСериал «Матрица» — это история о том, как люди в погоне за наживой убивают друг друга и спасают мир от неминуемой гибели.'

# Essays:

In [ ]:
!wget https://raw.githubusercontent.com/buvanenko/finetune_rugpt3/main/essays.txt

--2022-04-04 08:04:54--  https://raw.githubusercontent.com/buvanenko/finetune_rugpt3/main/essays.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1821921 (1.7M) [text/plain]
Saving to: ‘essays.txt’

essays.txt          100%[===================>]   1.74M  --.-KB/s    in 0.01s   

2022-04-04 08:04:54 (155 MB/s) - ‘essays.txt’ saved [1821921/1821921]



In [ ]:
data_path = "essays.txt"

In [ ]:
import numpy as np
import random
import nltk

with open(data_path, "r", encoding="utf-8") as file:
    text = file.read()

valid_size = 5

topics = []
all_essays = []
for line in text.split("</s>"):
    if "Тема:" in line and "Сочинение:" in line:
        essay_text = line.split("Сочинение:")
        if len(essay_text) == 2:
            topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = f"Сочинение: {essay_text}"
            essay_res = f"<s>{topic}\n{essay_text}</s>"
            all_essays.append(essay_res)
            topics.append(topic)

random.seed(1234)
np.random.seed(1234)

unique_topics = list(set(topics))

valid_topics = []

for _ in range(valid_size):
    # Use randint for more speed (on big lists it is faster)
    idx = np.random.randint(0, len(unique_topics))
    valid_topics.append(unique_topics[idx])

train = []
valid = []
for topic, essay in zip(topics, all_essays):
    is_train = True
    for valid_topic in valid_topics:
        if (
            nltk.edit_distance(valid_topic, topic[:len(valid_topic)]) < 20 or
            nltk.edit_distance(valid_topic[:len(topic)], topic) < 20 or
            nltk.edit_distance(valid_topic[len(topic):], topic) < 20 or
            nltk.edit_distance(valid_topic, topic[len(valid_topic):]) < 20
            ):
            is_train = False
    if is_train:
        train.append(essay)
    else:
        valid.append(essay)

print(f'{len(valid)}, {len(train)}')

with open("train.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(train))

with open("valid.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(valid))

400, 0


In [ ]:
train

[]

In [ ]:
valid

In [ ]:
valid[1]

In [ ]:
!python pretrain_transformers.py \
    --output_dir=essays_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

04/04/2022 07:56:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
04/04/2022 07:56:32 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp_0u8g0zp
Downloading: 100% 608/608 [00:00<00:00, 572kB/s]
04/04/2022 07:56:32 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/config.json in cache at /root/.cache/torch/transformers/767f65149d3a8095b8ac0370c3b2bbc7f05d863ff413b010d2f5206de0349fd0.1ede3f500f8b09312434582244c28d34f008436c5a38bb3afa159d2b38f8e758
04/04/2022 07:56:32 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/767f65149d3a8095b8ac0370c3b2bbc7f05d863ff413b010d2f5206de0349fd0.1ede3f500f8b0931243458

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=500

04/04/2022 07:57:33 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
Traceback (most recent call last):
  File "generate_transformers.py", line 268, in <module>
    main()
  File "generate_transformers.py", line 203, in main
    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
  File "/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py", line 393, in from_pretrained
    return cls._from_pretrained(*inputs, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py", line 496, in _from_pretrained
    list(cls.vocab_files_names.values()),
OSError: Model name 'essays_model' was not found in tokenizers model name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). We assumed 'essays_model' was 

In [ ]:
<s>Тема: 
<s>Тема: "Власть, лишенная авторитета, хуже, чем явное безвластие".\nСочинение: В</s>



# Train RuGPT3 on small dataset (УСПЕШНО)

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgraded, 21 newly installed, 0 to remove and 39 not upgraded.
Need to get 85.9 MB of archives.
After this operation, 511 MB of additional d

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 16 kB/s 
     |████████████████████████████████| 12.8 MB 31.7 MB/s 
     |████████████████████████████████| 7.6 MB 4.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [ ]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 9494, done.
remote: Counting objects: 100% (565/565), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 9494 (delta 311), reused 343 (delta 165), pack-reused 8929
Receiving objects: 100% (9494/9494), 14.80 MiB | 21.71 MiB/s, done.
Resolving deltas: 100% (6424/6424), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3

In [ ]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3 MB 5.6 MB/s 
     |████████████████████████████████| 1.1 MB 30.0 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 2.9 MB 33.4 MB/s 


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 4.70 MiB/s, done.
Resolving deltas: 100% (410/410), done.


### Prepare train data

In [ ]:
!echo train.txt > train.list
!echo valid.txt > valid.list

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/pretrain_gpt3.py \
  --train-data-path "train.list" \
  --test-data-path "valid.list" \
  --max-files-per-process 100 \
  --logging-dir="log" \
  --save model \
  --load-huggingface sberbank-ai/rugpt3small_based_on_gpt2 \
  --save-interval 1000 \
  --log-interval 100 \
  --eval-interval 1000 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --train-iters 2000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004

using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
Pretrain GPT3 model
arguments:
  attention_dropout ............ 0.1
  num_attention_heads .......... 12
  hidden_size .................. 768
  intermediate_size ............ None
  num_layers ................... 12
  layernorm_epsilon ............ 1e-05
  hidden_dropout ............... 0.1
  max_position_embeddings ...... 2048
  vocab_size ................... 30522
  deep_init .................... False
  make_vocab_size_divisible_by . 8
  cpu_optimizer ................ False
  cpu_torch_adam ............... False
  sparse_mode .................. all
  fp16 ......................... False
  fp32_embedding ............... False
  fp32_layernorm ............... False
  fp32_tokentypes .............. False
  fp32_allreduce ............... False
  hysteresis ................... 2
  loss_scale ................... None
  loss_scale_window ............ 1000
  min_scale .....

Источники с примерами кода:
* https://developers.sber.ru/portal/products/rugpt-3
* https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2
* https://github.com/sberbank-ai/ru-gpts/blob/master/examples/Finetune_and_generate_RuGPTs_only_with_megatron.ipynb
* https://github.com/sberbank-ai/ru-gpts/blob/master/examples/Generate_text_with_RuGPTs_HF.ipynb

Про гиперпараметры GPT-3 small:
* https://habr.com/ru/post/589663/

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/generate_samples.py \
  --load model/ \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --distributed-backend "nccl" \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim

Generate Samples
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done, size 50264
building GPT3 model ...
 > number of parameters on model parallel rank 0: 125231616
Load checkpoint from model/
global rank 0 is loading checkpoint model/iter_0002000/mp_rank_00/model_optim_rng.pt
  successfully loaded model/iter_0002000/mp_rank_00/model_optim_rng.pt
Loaded

Context prompt (stop to exit) >>> <s>Отзыв на фильм Рекрут:
<s>Отзыв на фильм Рекрут:

Taken time 28.83


Context: <s>Отзыв на фильм Рекрут:

GPT: <s>Отзыв на фильм Рекрут: Джеймс Клэйтон - талантливый программист, вынужденно подрабатывающий барменом, пытается узнать правду о таинственном исчезновении своего отца. Однажды с ним знакомится некто по имени Уолтер Бёрк, который является

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/convert2huggingface.py \
  --load model/ \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface model_hf

using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
prepare tokenizer done, size 50264
building GPT3 model ...
 > number of parameters on model parallel rank 0: 125231616
Load checkpoint from model/
global rank 0 is loading checkpoint model/iter_0002000/mp_rank_00/model_optim_rng.pt
  successfully loaded model/iter_0002000/mp_rank_00/model_optim_rng.pt
Loaded
Export to huggingface model  model_hf with config {'vocab_size': 50264, 'n_positions': 2048, 'n_ctx': 2048, 'n_embd': 768, 'n_layer': 12, 'n_head': 12}
Saved huggingface model <class 'src.model.distributed.DistributedDataParallel'>
Exported in huggingface format to model_hf


In [ ]:
!ls model_hf

config.json  pytorch_model.bin	      tokenizer_config.json
merges.txt   special_tokens_map.json  vocab.json


In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model = GPT2LMHeadModel.from_pretrained("model_hf")

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model_1 = load_tokenizer_and_model("/content/model_hf")

In [ ]:
generated = generate(model_1, tok, "Отзыв на фильм Рекрут:", num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated

['Отзыв на фильм Рекрут: Джеймс Клэйтон - талантливый программист, вынужденно подрабатывающий барменом, пытается узнать правду о таинственном исчезновении своего отца. Однажды с ним знакомится некто по имени Уолтер Бёрк, который']

In [ ]:
generated

['Тебе понравился фильм Рекрут? Именно так и решил для себя в последнюю минуту, юный Джеймс Клейтон, который уже не первый год безнадёжно тонет в попытках найти истинную причину гибели своего отца. И вот он - шанс. Неожидан']

In [ ]:
%cp -av '/content/model_hf' '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model'

'/content/model_hf' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf'
'/content/model_hf/tokenizer_config.json' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/tokenizer_config.json'
'/content/model_hf/special_tokens_map.json' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/special_tokens_map.json'
'/content/model_hf/vocab.json' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/vocab.json'
'/content/model_hf/merges.txt' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/merges.txt'
'/content/model_hf/config.json' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/config.json'
'/content/model_hf/pytorch_model.bin' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/test_model/model_hf/pytorch_model.bin'
